# Meeting Notebook for August 29, 2025

## Preliminary Notes:
From the August 21 meeting, we established that a transformer model on the aggregated data with a dashboard alone was not sufficient because the amount of data was very small. Instead, we have pivoted to an LSTM/Transformer/DLinear model comparison with the following changes:
- Consider a collection of neighboring counties. Grab the environmental variables from each of the other counties and use that data as extra columns in the feature vector.
- What is the definition of neighboring counties? Well, we want to stretch this definition to three possible interpretations: 
  1. Contiguous connection
  2. All of the central valley counties
  3. All counties surrounding the central valley
- From a recent paper, it seems rats and other rodents are carriers for Coccidioidomycosis. So look into getting rat spray data
- Eventually convert this into a better dashboard (stretch goal)


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import sys 
import os 

sys.path.append(os.path.abspath('..'))

In [3]:
# Set up the data

all_county_df = pd.read_csv("../../data/county_adjacency_list.csv")

In [4]:
all_county_df.head()

,County Name,County GEOID,Neighbor Name,Neighbor GEOID
0,"Autauga County, AL",1001,"Autauga County, AL",1001
1,"Autauga County, AL",1001,"Chilton County, AL",1021
2,"Autauga County, AL",1001,"Dallas County, AL",1047
3,"Autauga County, AL",1001,"Elmore County, AL",1051
4,"Autauga County, AL",1001,"Lowndes County, AL",1085


In [12]:
adj_map = (
  # first remove the repeat of county_name and neighbor_name, as every county is its own
  # neighbor
  all_county_df[all_county_df['County Name'] != all_county_df['Neighbor Name']]

  # you want to groupby the county_name but then select the neighbors
  .groupby('County Name')['Neighbor Name']

  # sort, remove repeats, and remove any NAs
  .apply(lambda s_: sorted(s_.dropna().unique()))

  # Convert to a dict for easy searching
  .to_dict()
)

In [11]:
adj_map["Fresno County, CA"]

['Inyo County, CA',
 'Kings County, CA',
 'Madera County, CA',
 'Merced County, CA',
 'Mono County, CA',
 'Monterey County, CA',
 'San Benito County, CA',
 'Tulare County, CA']